<a href="https://colab.research.google.com/github/joycejhyu/DS4A/blob/master/DS4A_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import spacy 
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Import the dataset

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
reviews = pd.read_csv("/content/drive/My Drive/amazon_reviews/reviews_cleaned.csv")

In [6]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
0,4.0,True,2015-11-14,A2J0S1IC4PU9G8,B0002ZW5UQ,"So, if you're bothering to read reviews about ...","Not bad, Toppik, but let's be clear about a fe...",2995.0,"{'Size:': ' 0.42 oz.', 'Color:': ' Dark Brown'}",120.077389,1.000000,1.0
1,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000FOI48G,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2721.0,NaN,100.249017,1.000000,1.0
2,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000GLRREU,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2713.0,{'Size:': ' Ultra'},99.953594,0.997058,1.0
3,4.0,True,2012-11-08,A1IGCUY11IG8I2,B00HX27AM4,"Everyone has different experiences, but the Fl...",Works well for me.,2378.0,NaN,95.284234,0.793852,1.0
4,5.0,True,2015-07-23,A1VEOLBPJTX620,B00DLGPQK4,So let me say three things that you should kno...,Works amazing for Brazilian/Underarm Use!!!,2056.0,{'Color:': ' Fuchsia'},82.345180,0.686268,1.0


In [7]:
import random

ind = random.choices([True, False], k= len(reviews))
not_ind = [not i for i in ind]

In [129]:
model_data = reviews[ind]
dashboard_data = reviews[not_ind]

KeyError: ignored

In [9]:
model_data.to_csv('model_data.csv', index=False)
dashboard_data.to_csv('dashboard_data.csv', index = False)

In [4]:
model_data = pd.read_csv('/content/drive/My Drive/amazon_reviews/model_data.csv')
dashboard_data = pd.read_csv('/content/drive/My Drive/amazon_reviews/dashboard_data.csv')

In [5]:
len(model_data)

60336

In [6]:
len(dashboard_data)

60876

In [7]:
model_data.vote.value_counts()

2.0      23164
3.0      11567
4.0       6594
5.0       4362
6.0       2766
         ...  
607.0        1
198.0        1
110.0        1
271.0        1
240.0        1
Name: vote, Length: 262, dtype: int64

In [13]:
model_data.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
0,4.0,True,2015-11-14,A2J0S1IC4PU9G8,B0002ZW5UQ,"So, if you're bothering to read reviews about ...","Not bad, Toppik, but let's be clear about a fe...",2995.0,"{'Size:': ' 0.42 oz.', 'Color:': ' Dark Brown'}",120.077389,1.000000,1.0
1,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000FOI48G,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2721.0,NaN,100.249017,1.000000,1.0
4,5.0,True,2015-07-23,A1VEOLBPJTX620,B00DLGPQK4,So let me say three things that you should kno...,Works amazing for Brazilian/Underarm Use!!!,2056.0,{'Color:': ' Fuchsia'},82.345180,0.686268,1.0
5,5.0,True,2012-09-21,AZ4EUUJVC8358,B00HX27AM4,I got one area of my body that I want to get r...,The truth - 11 months later,1918.0,NaN,76.799872,0.640160,1.0
6,5.0,True,2007-12-01,A2JCJJNY43QQIV,B000GLRREU,I've always prided myself on my good teeth. I ...,Waterpik first timer (for the product AND peri...,1341.0,{'Size:': ' Ultra'},49.288555,0.492460,1.0


In [8]:
tot_sent = model_data.overall.value_counts()[:4].sum() #51638 reviews to train our sentiment analysis
tot_pred = model_data.overall.value_counts()[3:].sum() #8698 reviews to predict the sentiment

In [9]:
sent_data = model_data[(model_data['sentiment']==1)| (model_data['sentiment']==0)] #sentiment analysis data
pred_data = model_data[model_data['sentiment'].isnull()] #prediction data

# 2. Preprocessing the reviews

In [10]:
# Merging reviewText and Summary

X = sent_data['reviewText'] + sent_data['summary']

In [11]:
#Clean
##Show random reviews to have an idea about the cleaning

rows = np.random.randint(0,len(X), size = 5)

for i in rows:
  print(X.iloc[i])

I like the ease of use in the pastilles. I make my own lotion and this is a base. It is essential in a great lotion.great
I am a licensed aesthetician, and love anything in Osmosis, as I believe it is probably one of the best product lines on the market. I actually bought this for my massage therapist for her post-inflammatory hyperpigmentation due to acne, and she is very dark skinned...she called me in 2 weeks and sent me a photo and the results were AMAZING!!! Almost ALL her dark spots were gone and getting better every day...she is sold! She said NOTHING had ever worked this well...great stuff!Another great product from Osmosis
I receive compliments on how nice my hair smells when I use this shampoo.  My hair feels very clean and even a day later doesn't feel like it is oily.  I am very happy with this.  I will be ordering more.I receive compliments on how nice my hair smells when I use this shampoo
Love the davines line I use a lot of their hair care and styling products! I love t

In [12]:
#remove ascii characters, html string and links
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

X = pd.DataFrame(X, columns = ['reviews'])
X = X.fillna('None')
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))', '', x, flags=re.MULTILINE))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'w+.[a-zA-Z]+.com.[a-zA-Z]+.[a-zA-Z0-9]+..', '', x))
X['reviews'] = X['reviews'].apply(lambda x: cleanhtml(x))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'\'', '-hap-', x))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'\\-hap-', '', x))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'-hap-', ' ', x))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'\&', 'and', x))
X['reviews'] = X['reviews'].apply(lambda x: re.sub(r'\n', '', x))

# 3. Sentiment Analysis

In [13]:
y = sent_data.sentiment
y = y.values.ravel()
y.shape

(51638,)

In [14]:
X = X.to_numpy()
X.shape

(51638, 1)

In [15]:
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

In [16]:
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()

# To build a list of stop words for filtering. We keep 'not' as it is a good indicator of sentiment.
stopwords = list(STOP_WORDS)
stopwords.remove('not')

In [17]:
import string
from spacy.lang.en import English

punctuations = string.punctuation

# Creating a Spacy Parser
parser = English()

def my_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations]
    return mytokens

X_string = [' '.join([str(i) for i in j])  for j in X_train]
#X_doc = [nlp(i) for i in X_string]

In [18]:
X_test_string = [' '.join([str(i) for i in j])  for j in X_test]

In [20]:
#Custom transformer using spaCy 
from sklearn.base import TransformerMixin 

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [21]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

vectorizer = TfidfVectorizer(tokenizer = my_tokenizer, ngram_range=(1,2), min_df = 4, max_features = 2000) #Trying tfidf with unigram and bigram
SVC_classifier = LinearSVC()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', SVC_classifier)])

#X_pp = vectorizer.fit_transform(X_string)

#Basic model
#SVC_score = cross_val_score(SVC_classifier, X_pp, y_train, cv = 5)
#print('The score of the SVC model is:', np.mean(SVC_score))

In [22]:
# Fit our data
pipe.fit(X_string,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fe5518918d0>),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=2000,
                                 min_df=4, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_wor...
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function my_tokenizer at 0x7fe55cb5d730>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight

In [23]:
# Predicting with a test dataset
y_pred = pipe.predict(X_test_string)

In [24]:
# Prediction Results
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 1949   508]
 [  279 10174]]
0.9390395042602634


In [30]:
results = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_pred)], axis = 1)

In [31]:
results.columns = ['reviewsum', 'pred_sent']
results.head()

,reviewsum,pred_sent
0,I love this product and use it often with good...,1.0
1,Great for soaking your retainer inGreat for so...,1.0
2,"Works very well, and I put it on at night. Wak...",1.0
3,Love this! I buy it all the time from Amazon i...,1.0
4,This product feels really good on the skin . M...,1.0


In [32]:
for i in np.random.randint(0, len(results), size = 5):
  print(results.loc[i, 'reviewsum'], '\n', results.loc[i, 'pred_sent'], '\n')

Perfect product for my fine very curly hair. Control but not a lot of crunch.Control but no crunch! 
 1.0 

I don t use this daily like it recommends because it s an exfoliater, and it s not good to exfoliate your skin every day.  I use it twice a week at night in place of my other cleanser (Image Skincare Ormedic Balancing Facial Cleanser), and follow with other Image Skincare products.  I think this is a great exfoliater because it doesn t irrate my skin at all, nor does it dry it out.  And no breakouts!Great product. 
 1.0 

If I m being completely honest, when I purchased Booty Magic I was skeptical, although I had high hopes that the product would do what it claims it does. I am three weeks in and can already see changes! I can t wait to see what my bum looks like in 60 days!!!From Flat to Plump 
 1.0 

I ve done the "variety pack" of blades, and I find Astra blades to work best for my skin.  Been using them for years with my safety razor, and have probably saved hundreds (yes, HU

# 4. Predict the sentiment for the 2/3 star reviews of model_data

In [33]:
X_new = pred_data['reviewText'] + pred_data['summary']

X_new = pd.DataFrame(X_new, columns = ['reviews'])
X_new = X_new.fillna('None')
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))', '', x, flags=re.MULTILINE))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'w+.[a-zA-Z]+.com.[a-zA-Z]+.[a-zA-Z0-9]+..', '', x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: cleanhtml(x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'\'', '-hap-', x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'\\-hap-', '', x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'-hap-', ' ', x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'\&', 'and', x))
X_new['reviews'] = X_new['reviews'].apply(lambda x: re.sub(r'\n', '', x))

X_new = X_new.to_numpy()
X_new_string = [' '.join([str(i) for i in j])  for j in X_new]
y_new_pred = pipe.predict(X_new_string)

In [34]:
X_new_string[0]

'I m not sure why they don t refer to this Waterpik with its model number, WP-100. That s what it is. (they ve since changed the name from  professional  to WP-100)I have used Waterpiks over the years, and finally broke down an bought another after years of doing without. As far as their usefulness as a dental device, I think they re great. My dentist doesn t think that highly of them, but after paying him $5000 for dental surgery, maybe he just wants to assure himself of continued business.I don t floss much, and I m not that dedicated toothbrusher either. So when I use the pik, and there are always some food particles dislodged by it, even after flossing, I figure it must be doing some good.As far as this model goes, it works as expected, but I don tr find any improvments over the model I had 30 years ago. From the picture it looks bigger. So it has a smaller footprint than I expected, but the reservoir is also quite small. It s just big enough for a cursory cleaning without having t

In [35]:
results_23 = pd.concat([pd.DataFrame(X_new), pd.DataFrame(y_new_pred)], axis = 1)
results_23.columns = ['reviewsum', 'pred_sent']

for i in np.random.randint(0, len(results_23), size = 5):
  print(results_23.loc[i, 'reviewsum'], '\n', results_23.loc[i, 'pred_sent'], '\n')

I ve only been using this for a month and it s really drying my hair out and making it strawlike. I just bought a different brand to make sure it s the flat iron and my new flat iron is much kinder to my hair (T3). Maybe it s not right for the kind of hair I have (very thick and somewhat wavy). Not sure, but I won t be using it anymore.Dried my hair and made it strawlike. 
 0.0 

Not impressed with this body butter.  The packaging is styrofoam, the butter is the consistency of hard soap and, no noticeable scent of patchouli.  Heavy, sweet vanilla and lavender.  A little lasts a looooong time.  Not for me.Where s the patchouli 
 1.0 

Very powerful hairdryer,does not have low enough setting for finer hair. Dries hair very quickly, more of a professional dryer. It was blowing hair all over, difficult to controldoes not have low enough setting for finer hair. Dries hair very quickly 
 1.0 

I was reallly expecting to love this after reading all the good reviews. However, this mascara flak

In [36]:
results_23.head()

,reviewsum,pred_sent
0,I m not sure why they don t refer to this Wate...,1.0
1,I m not sure why they don t refer to this Wate...,1.0
2,"Nioxin Cleanser, System 1 (Fine/Untreated/Norm...",1.0
3,I purchased this unit after seeing the infomer...,0.0
4,"Super moisturizing, a great toner and smells g...",1.0


In [37]:
results_23.pred_sent.value_counts()

1.0    5657
0.0    3055
Name: pred_sent, dtype: int64

In [38]:
neg_rev = results_23[results_23.pred_sent == 0]

In [39]:
for i in np.random.randint(0,len(neg_rev), size = 5):
  print(neg_rev.reviewsum.iloc[i])

I didn t end up loving the colors as much as i thought since this looks much more matte in the pictures than in reality. I hadnt expected this palette to be as incredibly shimmery and glitzy as it turned out to be. Its for sure not work appropriate as every single color is slightly  two-toned" depending on how the light hits it and one of the colors actually has chunky glitter in it.Not the same as the pictures
My favorite mascara was discontinued by the manufacturer, and I thought I d try stila Huge Extreme Lash Mascara.  I was very disappointed.  The mascara clumps horribly and is very "gloppy" because the brush picks up  way too much product, creating a harsh and unnatural look when applied to my lashes without wiping the brush first.  I use a tissue to remove all the excess mascara from the brush, apply what s left, and then the result is very nice.  But there is a tremendous waste of mascara -- this mascara is too expensive to have most of it wiped off on a tissue.Would not recomm

# 5. Improving with SMOTE

In [40]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm_string = vectorizer.fit_transform(X_string)

X_sm, y_sm = smote.fit_sample(X_sm_string, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [41]:
from imblearn.pipeline import make_pipeline

SMOTE_pipeline = make_pipeline(vectorizer, SMOTE(random_state=123, ratio='minority'), SVC_classifier) 

In [42]:
SMOTE_pipeline.fit(X_string, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=2000,
                                 min_df=4, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token...
                       m_neighbors='deprecated', n_jobs=1,
                       out_step='deprecated', random_state=123,
                       ratio='minority', sampling_strategy='minority',
                       svm_estimator='deprecated')),
                ('linearsvc',
                 LinearSV

In [51]:
y_sm_pred = SMOTE_pipeline.predict(X_test_string)

In [52]:
confusion_matrix(y_test, y_sm_pred)

array([[2080,  288],
       [ 630, 9994]])

In [59]:
print(confusion_matrix(y_test, y_pred))

[[ 1946   422]
 [  264 10360]]


In [53]:
accuracy_score(y_test, y_sm_pred)

0.9293411330049262

In [54]:
print(classification_report(y_test, y_sm_pred))

              precision    recall  f1-score   support

         0.0       0.77      0.88      0.82      2368
         1.0       0.97      0.94      0.96     10624

    accuracy                           0.93     12992
   macro avg       0.87      0.91      0.89     12992
weighted avg       0.93      0.93      0.93     12992



In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85      2368
         1.0       0.96      0.98      0.97     10624

    accuracy                           0.95     12992
   macro avg       0.92      0.90      0.91     12992
weighted avg       0.95      0.95      0.95     12992



In [55]:
#Test on 2/3 reviews

y_new_sm_pred = SMOTE_pipeline.predict(X_new_string)

results_23_sm = pd.concat([pd.DataFrame(X_new), pd.DataFrame(y_new_sm_pred)], axis = 1)
results_23_sm.columns = ['reviewsum', 'pred_sent']

for i in np.random.randint(0, len(results_23_sm), size = 5):
  print(results_23_sm.loc[i, 'reviewsum'], '\n', results_23_sm.loc[i, 'pred_sent'], '\n')

I have used an inexpensive base coat for years and have never had a problem.  I recently purchased the OPI because I was buying some other OPI products.  However, the first time I went to change my nail enamel after using the OPI, I realized that my nails were stained by my nail enamel (also OPI).  I have always used a base coat and have never had my nails stain until this time.  I ll go back to the cheap stuff.I have used an inexpensive base coat for years and ... 
 1.0 

This actually worked really well for me, but only for about 3 weeks and then it stopped turning on completely.This actually worked really well for me, but only ... 
 1.0 

This feels nice and I was going to re-order, but I noticed Alpha Hydrox is offering a new one, under the name of Alpha.  The new one is SPF30, so I ordered that instead.  I guess they are changing their whole line over to this new name?  This one is offered in a 3 oz. tube, which will last a lot longer than the old 1 oz. tube.This feels nice and I 

In [56]:
neg_rev_sm = results_23_sm[results_23_sm.pred_sent == 0]

for i in np.random.randint(0,len(neg_rev), size = 5):
  print(neg_rev.reviewsum.iloc[i])

Product is not as described, clips are not 2 inches and were sent loose and not in a bag.Product is not as described, clips are not 2 ...
so far after a month there has been no change with this product, though i have not experienced the rash that others have talked about, i have not seen the changes that others have talked about.did not work
It came in a different bottle as shown and the quality is different than when I have bought it in health food stores in the past.  Im wondering if this is a copy of the original company or if the packaging and quality has changed.  I won t be ordering again.It came in a different bottle as shown and the ...
3 stars because Ive only used it twice, therefor cannot give a fair rating in either direction yet. Unfortunately, Ive only used it twice because the strong plastic/rubber/chemical-ish smell transfers to your hair when using this brush and its all I smell in my hair all day. Ive tried washing it numerous times with shampoo, soap, etc to no avail

In [58]:
for i in np.random.randint(0,len(neg_rev), size = 5):
  print(neg_rev.reviewsum.iloc[i])

too big for short hair and 1 roller doesn t work, found out too late to send backThree Stars
It was okay but I ended up returning it because it is not the original babyliss titanium miracurl.Three Stars
I had received a sample of this elsewhere and really liked it. However, the product is a different than the sample, despite the name being the same. This full size product did not deliver anything special. It does not do any harm, but it does nothing for my skin around my eyes either.Neutral review, not worth the price
I first started using the Clarisonic about a year ago and saw some results using the sensitive skin brush head. I don t necessarily consider my skin as sensitive because it can withstand just about anything and it never gets irritated. Well after using the deep pore facial brush I thought I would see incredible results, but I was wrong. At first I thought it was because I was pressing hard on my face with it, but in reality it s just too abrasive on my face. I am having t

# 6. Improving results with biased SVM

In [70]:
from sklearn.linear_model import SGDClassifier

biased_SVM = SGDClassifier()
biased_SMOTE_pipeline = make_pipeline(vectorizer, SMOTE(random_state=123, ratio='minority'), biased_SVM) 

In [71]:
biased_SMOTE_pipeline.fit(X_string, y_train)

y_biased_pred = biased_SMOTE_pipeline.predict(X_test_string)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [72]:
confusion_matrix(y_test, y_biased_pred)

array([[2086,  282],
       [ 665, 9959]])

In [73]:
accuracy_score(y_test, y_biased_pred)

0.9271089901477833

In [64]:
print(classification_report(y_test, y_biased_pred))

              precision    recall  f1-score   support

         0.0       0.76      0.87      0.82      2368
         1.0       0.97      0.94      0.96     10624

    accuracy                           0.93     12992
   macro avg       0.87      0.91      0.89     12992
weighted avg       0.93      0.93      0.93     12992



# 7. Predicting for the Dashboard data

In [130]:
dashboard_data = pd.read_csv('dashboard_data.csv')

dashboard_data.head()
#ind = dashboard_data[dashboard_data.sentiment.isna()].index
#ind

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
0,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000GLRREU,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2713.0,{'Size:': ' Ultra'},99.953594,0.997058,1.0
1,4.0,True,2012-11-08,A1IGCUY11IG8I2,B00HX27AM4,"Everyone has different experiences, but the Fl...",Works well for me.,2378.0,NaN,95.284234,0.793852,1.0
2,5.0,True,2016-04-28,A28UB4TPSLLJOP,B001UHN0JA,"This is long but important. I have Very thick,...",Thick hair BLESSING,1005.0,"{'Size:': ' 1-""'}",40.112431,0.335115,1.0
3,1.0,True,2016-05-29,A27IKPUCSAWDU2,B0017SWIU4,I have really nice skin. It's one of the thing...,Read It All for Great Skin!!!,893.0,NaN,35.611891,0.297695,0.0
4,5.0,True,2011-07-05,A1SOOEG76KFBPM,B00547H5U2,"Ive been reading great reviews, but users comp...",RE: Lost Suction... the solution,859.0,NaN,34.245655,0.286335,1.0


In [140]:
dash_pred = dashboard_data[dashboard_data.sentiment.isna()]

In [25]:
X_dash = dashboard_data['reviewText'] + dashboard_data['summary']

X_dash = pd.DataFrame(X_dash, columns = ['reviews'])
X_dash = X_dash.fillna('None')
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))', '', x, flags=re.MULTILINE))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'w+.[a-zA-Z]+.com.[a-zA-Z]+.[a-zA-Z0-9]+..', '', x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: cleanhtml(x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'\'', '-hap-', x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'\\-hap-', '', x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'-hap-', ' ', x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'\&', 'and', x))
X_dash['reviews'] = X_dash['reviews'].apply(lambda x: re.sub(r'\n', '', x))

X_dash = X_dash.to_numpy()
X_dash_string = [' '.join([str(i) for i in j])  for j in X_dash]
y_dash_pred = pipe.predict(X_dash_string)

In [27]:
len(y_dash_pred)

60876

In [28]:
dash_results = pd.concat([dashboard_data, pd.DataFrame(y_dash_pred, columns = ['sentiment_pred'])],axis = 1)

In [32]:
dash_results['sentiment'] = dash_results['sentiment'].fillna(dash_results['sentiment_pred'])
dash_results = dash_results.drop('sentiment_pred', axis = 1)
dash_results.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
0,4.0,True,2015-11-14,A2J0S1IC4PU9G8,B0002ZW5UQ,"So, if you're bothering to read reviews about ...","Not bad, Toppik, but let's be clear about a fe...",2995.0,"{'Size:': ' 0.42 oz.', 'Color:': ' Dark Brown'}",120.077389,1.000000,1.0
1,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000FOI48G,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2721.0,NaN,100.249017,1.000000,1.0
2,5.0,True,2015-07-23,A1VEOLBPJTX620,B00DLGPQK4,So let me say three things that you should kno...,Works amazing for Brazilian/Underarm Use!!!,2056.0,{'Color:': ' Fuchsia'},82.345180,0.686268,1.0
3,5.0,True,2007-12-01,A2JCJJNY43QQIV,B000GLRREU,I've always prided myself on my good teeth. I ...,Waterpik first timer (for the product AND peri...,1341.0,{'Size:': ' Ultra'},49.288555,0.492460,1.0
4,5.0,True,2007-12-01,A2JCJJNY43QQIV,B000FOI48G,I've always prided myself on my good teeth. I ...,Waterpik first timer (for the product AND peri...,1341.0,NaN,49.288555,0.492460,1.0


In [38]:
dash_results.to_csv('/content/drive/My Drive/amazon_reviews/dashboard_sentiment.csv', index = False)

In [134]:
pd.DataFrame(y_dash_pred, columns = ['sentiment_pred']).value_counts()

sentiment_pred
1.0               4542
0.0               4115
dtype: int64

In [135]:
dash_results.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment,sentiment_pred
33,2.0,True,2009-11-09,A3BMXVPSW75SJF,B000Q9AKVE,I had a Remington wet/dry that finally got out...,I want my Remington back,341.0,NaN,12.360683,0.124678,NaN,NaN
36,3.0,True,2017-03-25,A26BKB2A9SGL3L,B00ZYJL9W0,I have been using crepe erase every day for a ...,Crepe erase is so so,330.0,NaN,12.988639,0.109589,NaN,NaN
38,3.0,True,2009-04-16,A2NNUWPC9MKQ0H,B000B9ECAE,The Supersmile Whitening System does its job o...,"Works, but ....",323.0,{'Size:': ' Small'},12.707355,0.107250,NaN,NaN
47,2.0,True,2006-10-23,A3N3GNVHFZZ6N1,B000FOI48G,"Yes indeed, this water pick does all that it i...",The Case of the Mysterious Drooling WaterPik,281.0,NaN,10.145011,0.102611,NaN,NaN
51,3.0,True,2012-05-28,A2WFZWRYWLC20S,B0002ZW5UQ,I ordered my first can of Toppick Hair Buildin...,"""Better than what I have now""",270.0,NaN,10.577635,0.089542,NaN,NaN


In [108]:
dash_results['sentiment'] = dash_results['sentiment'].fillna(dash_results['sentiment_pred'])
dash_results = dash_results.drop(['sentiment_pred'], axis = 1)

In [103]:
ind_dash = [i for i in dash_results.index]
ind_ndash = [i for i in range(0, len(dashboard_data))]

all_ind = [False if i in ind_dash else True for i in range(0, len(dash_results))]

In [104]:
len(all_ind)

8657

In [105]:
len(dashboard_data)

60534

In [123]:
dashboard_data_final = dashboard_data.copy()
dashboard_data_final['sentiment'] = dashboard_data['sentiment'].fillna(dash_results['sentiment'], axis = 'index')

In [124]:
pd.set_option.display.max_rows=1000
dashboard_data.head()

AttributeError: ignored

In [120]:
dashboard_data_final.iloc[76]

overall                                                       1
verified                                                   True
reviewTime                                           2014-05-22
reviewerID                                       A2OUFWHCFPJJBR
asin                                                 B005YB36O2
reviewText    I thought I'd be safe buying from Amazon, but ...
summary                                             COUNTERFEIT
vote                                                        210
style                         {'Format:': ' Health and Beauty'}
vote_stan                                               8.16663
vote_norm                                             0.0694955
sentiment                                                     0
Name: 181, dtype: object

In [113]:
dash_results.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
76,2.0,True,2009-11-09,A3BMXVPSW75SJF,B000Q9AKVE,I had a Remington wet/dry that finally got out...,I want my Remington back,341.0,NaN,12.360683,0.124678,1.0
82,3.0,True,2017-03-25,A26BKB2A9SGL3L,B00ZYJL9W0,I have been using crepe erase every day for a ...,Crepe erase is so so,330.0,NaN,12.988639,0.109589,1.0
84,3.0,True,2009-04-16,A2NNUWPC9MKQ0H,B000B9ECAE,The Supersmile Whitening System does its job o...,"Works, but ....",323.0,{'Size:': ' Small'},12.707355,0.107250,1.0
106,2.0,True,2006-10-23,A3N3GNVHFZZ6N1,B000FOI48G,"Yes indeed, this water pick does all that it i...",The Case of the Mysterious Drooling WaterPik,281.0,NaN,10.145011,0.102611,1.0
115,3.0,True,2012-05-28,A2WFZWRYWLC20S,B0002ZW5UQ,I ordered my first can of Toppick Hair Buildin...,"""Better than what I have now""",270.0,NaN,10.577635,0.089542,1.0


In [117]:
dashboard_data_final.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,vote,style,vote_stan,vote_norm,sentiment
2,5.0,True,2011-10-24,A2LXX47A0KMJVX,B000GLRREU,"<div id=""video-block-R2HEG12C48KXPW"" class=""a-...","Very happy with this. Quiet operation, large w...",2713.0,{'Size:': ' Ultra'},99.953594,0.997058,1.0
3,4.0,True,2012-11-08,A1IGCUY11IG8I2,B00HX27AM4,"Everyone has different experiences, but the Fl...",Works well for me.,2378.0,NaN,95.284234,0.793852,1.0
8,5.0,True,2016-04-28,A28UB4TPSLLJOP,B001UHN0JA,"This is long but important. I have Very thick,...",Thick hair BLESSING,1005.0,"{'Size:': ' 1-""'}",40.112431,0.335115,1.0
10,1.0,True,2016-05-29,A27IKPUCSAWDU2,B0017SWIU4,I have really nice skin. It's one of the thing...,Read It All for Great Skin!!!,893.0,NaN,35.611891,0.297695,0.0
13,5.0,True,2011-07-05,A1SOOEG76KFBPM,B00547H5U2,"Ive been reading great reviews, but users comp...",RE: Lost Suction... the solution,859.0,NaN,34.245655,0.286335,1.0
